In [3]:
# ==============================
# YOLOv8 Test Inference & Metrics
# ==============================
from ultralytics import YOLO
from pathlib import Path

# 1) 모델 불러오기
WEIGHT = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250910_merge_data_yolov11s/weights/best.pt"
DATASET = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data_split"

model = YOLO(WEIGHT)

# 2) 테스트셋 평가 (metrics 출력)
results = model.val(
    data=str(Path(DATASET) / "data.yaml"),  # split 안에 data.yaml 있어야 함
    split="test",          # test셋으로 평가
    imgsz=1024,            # 이미지 사이즈
    batch=16,              # 배치 크기 (GPU 여유 없으면 줄이기)
    device=0               # GPU ID (없으면 "cpu")
)

# # 3) 개별 이미지 추론 결과 확인 (원한다면)
# out_dir = Path("runs_yolo/infer_test")
# pred = model.predict(
#     source=str(Path(DATASET) / "images" / "test"),
#     imgsz=1024,
#     conf=0.25,     # confidence threshold
#     save=True,
#     project=out_dir.parent,
#     name=out_dir.name,
#     device=0
# )

print("[INFO] 테스트셋 평가 완료")


Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
YOLO11s summary (fused): 100 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2266.8±2484.5 MB/s, size: 1191.4 KB)


val: Scanning /home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data_split/labels/test.cache... 314 images, 69 backgrounds, 0 corrupt: 100%|██████████| 383/383 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:05<00:00,  4.25it/s]


                   all        383       1156      0.808      0.694      0.819      0.675
                 Divot        274        558      0.821      0.803      0.852      0.684
           Fixed_Divot        207        464      0.823      0.834      0.883      0.711
         Dieased_Grass         37         66          1      0.213      0.598      0.505
                  Pole         18         18      0.642      0.722      0.702      0.404
             Sprinkler          5          5      0.516          1      0.995      0.995
                 Drain         22         22          1       0.77      0.897      0.735
             Golf ball         23         23      0.856      0.517      0.804      0.691
Speed: 1.0ms preprocess, 11.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val15
[INFO] 테스트셋 평가 완료


In [12]:
# ==========================
# Delete all .txt label files except classes.txt
# ==========================
from pathlib import Path
import os

folders = [
    "/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_bad_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_bad_data",
]

deleted = 0
for f in folders:
    path = Path(f)
    if not path.exists():
        print(f"[WARN] 경로 없음: {f}")
        continue

    for txt in path.rglob("*.txt"):
        # === classes.txt는 삭제하지 않음 ===
        if txt.name == "classes.txt":
            continue
        try:
            os.remove(txt)
            deleted += 1
        except Exception as e:
            print(f"[ERROR] 삭제 실패: {txt} ({e})")

print(f"[RESULT] 총 {deleted}개의 txt 파일 삭제 완료 (classes.txt 제외)")

[RESULT] 총 140개의 txt 파일 삭제 완료 (classes.txt 제외)


In [15]:
# === YOLOv8 자동 라벨링 (클래스별 서로 다른 confidence 적용 + 지정 순서로 저장) ===
from ultralytics import YOLO
from pathlib import Path
import cv2
import math

# ====== 경로 설정 ======
WEIGHT = Path("/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250911_merge_data_yolov8m/weights/best.pt")
IMG_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data")
IMG_EXTS = [".jpg", ".jpeg", ".png"]

# ====== 원하는 "저장용 클래스 순서" (요청하신 순서) ======
desired_order = [
    "Divot",
    "Fixed_Divot",
    "Diseased_Grass",  # 사용자가 제공한 철자 그대로 둠
    "Confused_Object",
    "Pole",
    "Sprinkler",
    "Drain",
    "Golf ball",
]

# ====== 클래스별 confidence threshold (원하면 자유롭게 조정) ======
# 키는 "클래스 이름" (위 desired_order 기준), 값은 임계값(0~1)
per_class_conf = {
    "Divot":            0.25,
    "Fixed_Divot":      0.25,
    "Diseased_Grass":    0.40,
    "Confused_Object":  0.50,
    "Pole":             0.50,
    "Sprinkler":        0.50,
    "Drain":            0.50,
    "Golf ball":        0.50,
}
DEFAULT_CONF = 0.5  # 사전에 없는 클래스 이름이 들어올 경우 기본값

# ====== 모델 로드 ======
model = YOLO(str(WEIGHT))
model_names = model.names  # ex) {0: 'Divot', 1: 'Fixed_Divot', ...} 또는 list
if isinstance(model_names, dict):
    idx2name = {int(k): v for k, v in model_names.items()}
else:
    idx2name = {i: n for i, n in enumerate(model_names)}

# ====== "모델 클래스 인덱스" -> "저장용 클래스 인덱스" 매핑 만들기 ======
# 1) 이름이 정확히 매칭되면 그 이름의 위치(desired_order index)로 매핑
# 2) 이름이 매칭 안 되면 경고를 남기고 "동일 인덱스"로 저장(최소한 깨지지 않게)
name2desiredidx = {n: i for i, n in enumerate(desired_order)}
modelidx_to_saveidx = {}
unmatched = []

for midx, mname in idx2name.items():
    if mname in name2desiredidx:
        modelidx_to_saveidx[midx] = name2desiredidx[mname]
    else:
        unmatched.append((midx, mname))
        modelidx_to_saveidx[midx] = midx  # fallback: 인덱스 그대로

if unmatched:
    print("[WARN] 아래 모델 클래스 이름은 desired_order와 매칭되지 않아 '동일 인덱스'로 저장합니다:")
    for midx, mname in unmatched:
        print(f"  - model idx {midx}: '{mname}'")

# ====== 이미지 수집 ======
img_files = [p for p in IMG_DIR.iterdir() if p.suffix.lower() in IMG_EXTS]
print(f"[INFO] Found {len(img_files)} images")

saved_box_count = 0
saved_per_class = {i: 0 for i in range(len(desired_order))}

for img_path in img_files:
    # 예측: 여기서는 conf를 낮게(예: 0.01) 주고, 필터링은 아래에서 클래스별로 수행
    results = model.predict(source=str(img_path), conf=0.01, save=False, verbose=False)

    boxes = results[0].boxes
    if boxes is None or len(boxes) == 0:
        continue

    img = cv2.imread(str(img_path))
    if img is None:
        print(f"[WARN] 이미지 열기 실패: {img_path}")
        continue
    h, w = img.shape[:2]

    # 라벨 파일 열기(필요할 때만 생성)
    lines = []
    for b in boxes:
        cls_idx = int(b.cls[0].item())
        conf = float(b.conf[0].item())

        # 모델 클래스 이름을 가져와 per-class threshold 선택
        cls_name = idx2name.get(cls_idx, None)
        # 저장용 클래스 인덱스(요청 순서 기준)
        save_cls_idx = modelidx_to_saveidx.get(cls_idx, cls_idx)

        # 클래스 이름 기반 임계값(없으면 기본값)
        thr = per_class_conf.get(cls_name, DEFAULT_CONF)

        if conf < thr:
            continue

        # xyxy -> xywh (normalized)
        x1, y1, x2, y2 = b.xyxy[0]
        cx = ((x1 + x2) / 2) / w
        cy = ((y1 + y2) / 2) / h
        bw = (x2 - x1) / w
        bh = (y2 - y1) / h

        # 클램프(숫자 오차 대비)
        def clamp(v): 
            return max(0.0, min(1.0, float(v)))
        cx, cy, bw, bh = map(clamp, (cx, cy, bw, bh))

        # 라벨 라인(요청: "저장용 클래스 순서"의 인덱스로 기록)
        lines.append(f"{save_cls_idx} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

        if 0 <= save_cls_idx < len(desired_order):
            saved_per_class[save_cls_idx] += 1
        saved_box_count += 1

    if lines:
        with open(img_path.with_suffix(".txt"), "w") as f:
            f.write("\n".join(lines))

# ====== 리포트 ======
print("====================================")
print(f"[RESULT] 총 이미지 수         : {len(img_files)}")
print(f"[RESULT] 저장된 라벨(박스) 수 : {saved_box_count}")
print("[RESULT] 클래스별 저장된 박스 수 (저장용 순서 기준):")
for i, name in enumerate(desired_order):
    print(f"  - {i:02d} {name:15s}: {saved_per_class.get(i,0)}")
print("====================================")
print("[NOTE] per-class threshold:", per_class_conf)

[INFO] Found 9690 images
[RESULT] 총 이미지 수         : 9690
[RESULT] 저장된 라벨(박스) 수 : 8777
[RESULT] 클래스별 저장된 박스 수 (저장용 순서 기준):
  - 00 Divot          : 1512
  - 01 Fixed_Divot    : 5979
  - 02 Diseased_Grass : 996
  - 03 Confused_Object: 0
  - 04 Pole           : 50
  - 05 Sprinkler      : 3
  - 06 Drain          : 143
  - 07 Golf ball      : 94
[NOTE] per-class threshold: {'Divot': 0.25, 'Fixed_Divot': 0.25, 'Diseased_Grass': 0.4, 'Confused_Object': 0.5, 'Pole': 0.5, 'Sprinkler': 0.5, 'Drain': 0.5, 'Golf ball': 0.5}


In [20]:
from pathlib import Path
import json
import re

# === 경로 설정 ===
JSON_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/label")
OUT_DIR  = JSON_DIR.parent / "label_txt"   # 결과 저장 폴더: /.../data/label_txt
OUT_DIR.mkdir(parents=True, exist_ok=True)

# === 클래스 매핑 (사용자 지정) ===
# 0 Divot, 1 Fixed_Divot, 2 Diseased_Grass, 3 Confused_Object,
# 4 Pole, 5 Sprinkler(오타 포함), 6 Drain, 7 Golf ball
CLASS_MAP = {
    "divot": 0,
    "fixed_divot": 1,
    "diseased_grass": 2,
    "confused_class": 3,       # 혹시 JSON에 이렇게 있을 경우 대비
    "Confused_Object": 3,      # 혹시 대문자 표기가 있다면 대비
    "Pole": 4,
    "sprinkler": 5,
    "springkler": 5,           # 오타도 5번
    "waterdrain": 6,
    "ball": 7,
}

def parse_wh_from_string(s: str):
    """
    문자열 s의 끝에서 '..._1920x1536' 형태를 찾아 (w,h) 반환.
    없으면 None.
    """
    m = re.search(r'(\d+)[xX](\d+)(?:\.\w+)?$', s)
    if m:
        return int(m.group(1)), int(m.group(2))
    return None

def get_image_wh(data: dict, json_path: Path):
    """
    1) JSON 내부 image_path의 끝에서 1920x1536 패턴을 먼저 시도
    2) 안되면 JSON 파일명(stem)의 끝에서 패턴 시도
    """
    # 1) image_path 사용
    img_path = data.get("image_path")
    if isinstance(img_path, str):
        wh = parse_wh_from_string(img_path)
        if wh: 
            return wh

    # 2) json 파일명에서 시도
    wh = parse_wh_from_string(json_path.stem)
    if wh:
        return wh

    raise ValueError(f"해상도(가로x세로)를 파일명/JSON에서 찾지 못했습니다: {json_path.name}")

def to_yolo_line(cls_id, x1, y1, x2, y2, w, h):
    # 좌표 정리
    x_min, x_max = sorted([x1, x2])
    y_min, y_max = sorted([y1, y2])
    xc = (x_min + x_max) / 2.0 / w
    yc = (y_min + y_max) / 2.0 / h
    bw = (x_max - x_min) / w
    bh = (y_max - y_min) / h
    return f"{cls_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}"

def extract_xyxy(coords):
    """
    coords가 [[x1,y1],[x2,y2]] 또는 {0:{0:x1,1:y1},1:{0:x2,1:y2}} 같은 형태일 가능성 대비.
    (사용 예시에 맞춰 유연 처리)
    """
    # 리스트/튜플로 들어오는 경우
    if isinstance(coords, (list, tuple)) and len(coords) >= 2:
        p0, p1 = coords[0], coords[1]
        x1, y1 = float(p0[0]), float(p0[1])
        x2, y2 = float(p1[0]), float(p1[1])
        return x1, y1, x2, y2

    # dict로 들어오는 경우 (예시처럼 0,1 키를 쓰는 구조)
    if isinstance(coords, dict) and 0 in coords and 1 in coords:
        p0, p1 = coords[0], coords[1]
        # p0, p1이 다시 dict로 0:x, 1:y 형태일 수 있음
        if isinstance(p0, dict) and 0 in p0 and 1 in p0 and isinstance(p1, dict) and 0 in p1 and 1 in p1:
            x1, y1 = float(p0[0]), float(p0[1])
            x2, y2 = float(p1[0]), float(p1[1])
            return x1, y1, x2, y2

    raise ValueError(f"좌표 형식을 해석하지 못했습니다: {coords}")

converted = 0
skipped = 0

for jf in JSON_DIR.glob("*.json"):
    try:
        with open(jf, "r") as f:
            data = json.load(f)

        W, H = get_image_wh(data, jf)

        lines = []
        for ann in data.get("annotations", []):
            subtype = ann.get("subtype")
            if not subtype:
                continue
            subtype_norm = str(subtype).strip()

            # 매핑되지 않은 클래스는 스킵
            if subtype_norm not in CLASS_MAP:
                skipped += 1
                continue

            cls_id = CLASS_MAP[subtype_norm]

            coords = ann.get("coordinates")
            if coords is None:
                skipped += 1
                continue

            x1, y1, x2, y2 = extract_xyxy(coords)
            lines.append(to_yolo_line(cls_id, x1, y1, x2, y2, W, H))

        # 빈 파일도 생성(원하면 조건을 바꿔서 skip 가능)
        out_name = OUT_DIR / (jf.stem + ".txt")
        with open(out_name, "w") as f:
            f.write("\n".join(lines))

        converted += 1

    except Exception as e:
        skipped += 1
        print(f"[WARN] {jf.name} 변환 실패: {e}")

print(f"[DONE] 변환 완료: {converted}개, 스킵/경고: {skipped}개")
print(f"출력 폴더: {OUT_DIR}")

[DONE] 변환 완료: 7155개, 스킵/경고: 0개
출력 폴더: /home/dw/ws_job_msislab/Golf_Project/data/20250904/label_txt


In [19]:
from pathlib import Path

# === 설정 ===
DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904_2/img")
IMG_EXTS = {".jpg", ".jpeg", ".png"}
PREFIX   = "20250904_"
PAD      = 5
DRY_RUN  = False   # True면 계획만 출력

def next_name(i: int) -> str:
    return f"{PREFIX}{i:0{PAD}d}"

# 이미지 파일 수집 (이름순 정렬)
images = sorted([p for p in DIR.iterdir()
                 if p.is_file() and p.suffix.lower() in IMG_EXTS])

if not images:
    print("[INFO] 이미지 없음")
    raise SystemExit

# 최종 매핑 (이미지 → 새 이름)
img_map = {}
for i, img in enumerate(images, start=1):
    new_stem = next_name(i)
    img_map[img] = img.with_name(new_stem + img.suffix.lower())

# 계획 출력
print("=== 리네임 계획 ===")
for old, new in img_map.items():
    print(f"{old.name} -> {new.name}")

if DRY_RUN:
    print("[DRY-RUN] 실제 변경 없음")
    raise SystemExit

# 충돌 방지: 임시 이름으로 1차 이동
def to_tmp(p: Path) -> Path:
    return p.with_name(f"__tmp__{p.name}")

tmp_map = {old: to_tmp(old) for old in img_map}
for old, tmp in tmp_map.items():
    if tmp.exists():
        tmp.unlink()
    old.rename(tmp)

# 임시 → 최종 이름으로 2차 이동
for tmp, final in zip(tmp_map.values(), img_map.values()):
    tmp.rename(final)

print(f"[DONE] 이미지 {len(img_map)}개 리네임 완료")


=== 리네임 계획 ===
front_raw_25-09-04_17-52-04_00000_000016_f000075_1920x1536.jpg -> 20250904_00001.jpg
front_raw_25-09-04_17-52-04_00000_000017_f000080_1920x1536.jpg -> 20250904_00002.jpg
front_raw_25-09-04_17-52-04_00000_000018_f000085_1920x1536.jpg -> 20250904_00003.jpg
front_raw_25-09-04_17-52-04_00000_000019_f000090_1920x1536.jpg -> 20250904_00004.jpg
front_raw_25-09-04_17-52-04_00000_000020_f000095_1920x1536.jpg -> 20250904_00005.jpg
front_raw_25-09-04_17-52-04_00000_000021_f000100_1920x1536.jpg -> 20250904_00006.jpg
front_raw_25-09-04_17-52-04_00000_000022_f000105_1920x1536.jpg -> 20250904_00007.jpg
front_raw_25-09-04_17-52-04_00000_000023_f000110_1920x1536.jpg -> 20250904_00008.jpg
front_raw_25-09-04_17-52-04_00000_000024_f000115_1920x1536.jpg -> 20250904_00009.jpg
front_raw_25-09-04_17-52-04_00000_000025_f000120_1920x1536.jpg -> 20250904_00010.jpg
front_raw_25-09-04_17-52-04_00000_000026_f000125_1920x1536.jpg -> 20250904_00011.jpg
front_raw_25-09-04_17-52-04_00000_000027_f000130_1

In [21]:
from pathlib import Path

# === 설정 ===
DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_origin")
IMG_EXTS = {".jpg", ".jpeg", ".png"}
LABEL_EXT = ".txt"
PREFIX = "20250904_"
PAD = 5
DRY_RUN = False  # True로 두면 계획만 출력

def next_name(i: int) -> str:
    return f"{PREFIX}{i:0{PAD}d}"

# 폴더 내 파일 수집
files = sorted([p for p in DIR.iterdir() if p.is_file()])
images = [p for p in files if p.suffix.lower() in IMG_EXTS]

if not images:
    print("[INFO] 이미지 없음")
    raise SystemExit

# (1) 최종 리네임 매핑 생성: 이미지 기준으로 순번 부여, txt 있으면 같이 매핑
img_map, txt_map = {}, {}
i = 1
for img in images:
    new_stem = next_name(i)
    new_img = img.with_name(new_stem + img.suffix.lower())
    img_map[img] = new_img

    lbl = img.with_suffix(LABEL_EXT)
    if lbl.exists() and lbl.name != "classes.txt":
        txt_map[lbl] = lbl.with_name(new_stem + LABEL_EXT)
    i += 1

# (2) 충돌 방지용 임시 이름으로 1차 이동 계획
def to_tmp(p: Path) -> Path:
    return p.with_name(f"__tmp__{p.name}")

tmp_img_map = {old: to_tmp(old) for old in img_map}
tmp_txt_map = {old: to_tmp(old) for old in txt_map}

# 계획 출력
print("=== 리네임 계획 ===")
for old, new in img_map.items():
    print(f"[IMG] {old.name} -> {new.name}")
for old, new in txt_map.items():
    print(f"[TXT] {old.name} -> {new.name}")

if DRY_RUN:
    print("[DRY-RUN] 실제 변경 없음")
    raise SystemExit

# (3) 1차: 임시 이름으로 이동 (충돌 회피)
for old, tmp in tmp_img_map.items():
    if tmp.exists():
        tmp.unlink()
    old.rename(tmp)

for old, tmp in tmp_txt_map.items():
    if tmp.exists():
        tmp.unlink()
    old.rename(tmp)

# (4) 2차: 임시에서 최종 이름으로 이동
for old_tmp, final in zip(tmp_img_map.values(), img_map.values()):
    old_tmp.rename(final)

for old_tmp, final in zip(tmp_txt_map.values(), txt_map.values()):
    old_tmp.rename(final)

print(f"[DONE] 이미지 {len(img_map)}개 리네임")
print(f"[DONE] 텍스트 {len(txt_map)}개 리네임 (없는 경우는 스킵됨)")


=== 리네임 계획 ===
[IMG] front_raw_25-09-04_17-52-04_00000_000016_f000075_1920x1536.jpg -> 20250904_00001.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000017_f000080_1920x1536.jpg -> 20250904_00002.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000018_f000085_1920x1536.jpg -> 20250904_00003.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000019_f000090_1920x1536.jpg -> 20250904_00004.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000020_f000095_1920x1536.jpg -> 20250904_00005.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000021_f000100_1920x1536.jpg -> 20250904_00006.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000022_f000105_1920x1536.jpg -> 20250904_00007.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000023_f000110_1920x1536.jpg -> 20250904_00008.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000024_f000115_1920x1536.jpg -> 20250904_00009.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000025_f000120_1920x1536.jpg -> 20250904_00010.jpg
[IMG] front_raw_25-09-04_17-52-04_00000_000026_f000125_1920x1536.jpg -> 202

In [6]:

from pathlib import Path

LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_data_not_use")

invalid_lines = {}  # {파일경로: [문제된 라인들]}

for txt_file in LABEL_DIR.rglob("*.txt"):
    with open(txt_file, "r") as f:
        lines = [line.strip() for line in f if line.strip()]
    bads = []
    for line in lines:
        parts = line.split()
        if len(parts) == 5:
            w, h = float(parts[3]), float(parts[4])
            if w == 0.0 and h == 0.0:   # 둘 다 0이면 무효
                bads.append(line)
    if bads:
        invalid_lines[txt_file] = bads

print("=== width와 height가 모두 0인 무효 박스 ===")
for path, bads in invalid_lines.items():
    print(path)
    for l in bads:
        print("   ", l)

print(f"\n총 {len(invalid_lines)}개 파일에서 무효 박스 발견")


=== width와 height가 모두 0인 무효 박스 ===

총 0개 파일에서 무효 박스 발견


In [7]:
from pathlib import Path

LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data")

for txt_file in LABEL_DIR.rglob("*.txt"):
    with open(txt_file, "r") as f:
        lines = [line.rstrip("\n") for line in f]

    # 폭이나 높이가 0인 줄 제거
    filtered = []
    for line in lines:
        parts = line.split()
        if len(parts) == 5:
            w, h = float(parts[3]), float(parts[4])
            if w == 0.0 or h == 0.0:
                continue  # 해당 줄 삭제
        filtered.append(line)

    # 실제로 삭제된 경우만 덮어쓰기
    if len(filtered) != len(lines):
        with open(txt_file, "w") as f:
            if filtered:                     # 남은 라인이 있으면
                f.write("\n".join(filtered) + "\n")
            else:                             # 모두 삭제된 경우 빈 파일로
                f.truncate(0)

print("완료: width 또는 height가 0.000000 인 라인을 모두 제거했습니다.")


완료: width 또는 height가 0.000000 인 라인을 모두 제거했습니다.


In [10]:
from pathlib import Path
import shutil

BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data")
DEST = BASE / "class3"
DEST.mkdir(exist_ok=True)

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

def has_class3(txt_path: Path) -> bool:
    with open(txt_path, "r") as f:
        for line in f:
            if line.strip() and line.split()[0] == "3":
                return True
    return False

def find_image(label_path: Path):
    stem = label_path.stem
    parent = label_path.parent
    for ext in IMG_EXTS:
        cand = parent / f"{stem}{ext}"
        if cand.exists():
            return cand
    return None

moved_txt = 0
moved_img = 0

for txt in BASE.rglob("*.txt"):
    if has_class3(txt):
        img = find_image(txt)

        # 라벨 이동
        shutil.move(txt, DEST / txt.name)
        moved_txt += 1

        # 이미지도 있으면 이동
        if img:
            shutil.move(img, DEST / img.name)
            moved_img += 1

print(f"[완료] class 3 라벨 {moved_txt}개, 이미지 {moved_img}개를 {DEST}로 이동했습니다.")


[완료] class 3 라벨 420개, 이미지 420개를 /home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data/class3로 이동했습니다.


In [9]:
from pathlib import Path

CLASS3_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data/class3")

# 확장자: 이미지와 txt 모두 처리
TARGET_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".txt",
               ".JPG", ".JPEG", ".PNG", ".BMP"}

renamed = 0

for f in CLASS3_DIR.iterdir():
    if f.suffix not in TARGET_EXTS:
        continue
    stem = f.stem
    # 이름이 _1로 끝나면
    if stem.endswith("_1"):
        new_name = stem[:-2] + f.suffix  # "_1" 제거
        new_path = f.with_name(new_name)
        if new_path.exists():
            print(f"[SKIP] {new_path} 이미 존재 → 덮어쓰지 않음")
            continue
        f.rename(new_path)
        renamed += 1
        print(f"RENAMED: {f.name} → {new_path.name}")

print(f"\n총 {renamed}개 파일에서 '_1'을 제거했습니다.")

RENAMED: 20250904_01536_1.jpg → 20250904_01536.jpg
RENAMED: 20250904_01534_1.txt → 20250904_01534.txt
RENAMED: 20250904_00149_1.jpg → 20250904_00149.jpg
RENAMED: 20250904_01108_1.jpg → 20250904_01108.jpg
RENAMED: 20250904_02944_1.txt → 20250904_02944.txt
RENAMED: 20250904_00260_1.txt → 20250904_00260.txt
RENAMED: 20250904_01545_1.jpg → 20250904_01545.jpg
RENAMED: 20250904_01217_1.txt → 20250904_01217.txt
RENAMED: 20250904_01292_1.jpg → 20250904_01292.jpg
RENAMED: 20250904_00257_1.jpg → 20250904_00257.jpg
RENAMED: 20250904_01291_1.txt → 20250904_01291.txt
RENAMED: 20250904_01257_1.txt → 20250904_01257.txt
RENAMED: 20250904_00231_1.jpg → 20250904_00231.jpg
RENAMED: 20250904_01129_1.jpg → 20250904_01129.jpg
RENAMED: 20250904_01246_1.txt → 20250904_01246.txt
RENAMED: 20250904_00590_1.txt → 20250904_00590.txt
RENAMED: 20250904_00596_1.jpg → 20250904_00596.jpg
RENAMED: 20250904_00265_1.jpg → 20250904_00265.jpg
RENAMED: 20250904_01400_1.jpg → 20250904_01400.jpg
RENAMED: 20250904_01106_1.jpg →